# Purpose

  1. Connect to your PostgreSQL DB.
  2. Fetch existing order data (either from staging.orders or dw.fact_orders).
  3. Generate a small set of synthetic warehouse data.
  4. Generate synthetic shipment records (one or more shipments per order) 
     with random carriers, dates, and costs.
  5. Insert results into staging.synthetic_warehouse and staging.synthetic_shipments.

In [3]:
import random
import datetime
import psycopg2
from psycopg2.extras import execute_batch

# Method 1: Direct DB Injection

In [33]:
#!/usr/bin/env python3
"""
generate_synthetic_shipment_data.py

Purpose:
  1. Connect to your PostgreSQL DB.
  2. Fetch existing order data (either from staging.orders or dw.fact_orders).
  3. Generate a small set of synthetic warehouse data.
  4. Generate synthetic shipment records (one or more shipments per order) 
     with random carriers, dates, and costs.
  5. Insert results into staging.synthetic_warehouse and staging.synthetic_shipments.

Usage:
  python generate_synthetic_shipment_data.py
"""

import random
import datetime
import psycopg2
from psycopg2.extras import execute_batch

# -----------------------------------------------------------------------------
# Configuration
# -----------------------------------------------------------------------------
DB_HOST = "localhost"
DB_NAME = "ecommerce_warehouse"
DB_USER = "postgres"
DB_PASSWORD = "YourPasswordHere"
DB_PORT = 5432

NUM_WAREHOUSES = 5  # How many synthetic warehouses to generate
CARRIERS = ["Correios", "Loggi", "Stark Logistics", "Dragonfly Express", "RapidAir"]


# -----------------------------------------------------------------------------
# Helper functions
# -----------------------------------------------------------------------------
def random_date(start_date, end_date):
    """
    Return a random datetime between two datetime objects.
    """
    delta = end_date - start_date
    rand_seconds = random.randrange(int(delta.total_seconds()))
    return start_date + datetime.timedelta(seconds=rand_seconds)

# -----------------------------------------------------------------------------
# Main Logic
# -----------------------------------------------------------------------------
def main():
    # 1. Connect to Postgres
    conn = psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        port=DB_PORT
    )
    conn.autocommit = True  # So each statement commits automatically
    cursor = conn.cursor()

    # 2. (Optional) Create staging tables for synthetic data if not exist
    #    staging.synthetic_warehouse, staging.synthetic_shipments
    create_staging_sql = """
    CREATE SCHEMA IF NOT EXISTS staging;

    CREATE TABLE IF NOT EXISTS staging.synthetic_warehouse (
        synthetic_warehouse_id SERIAL PRIMARY KEY,
        warehouse_name TEXT,
        warehouse_location TEXT,
        capacity INT
    );

    CREATE TABLE IF NOT EXISTS staging.synthetic_shipments (
        synthetic_shipment_id SERIAL PRIMARY KEY,
        order_id TEXT,
        warehouse_id INT,   -- will map to synthetic_warehouse_id
        carrier TEXT,
        ship_date TIMESTAMP,
        delivery_date TIMESTAMP,
        shipping_cost NUMERIC(10,2)
    );
    """
    cursor.execute(create_staging_sql)

    # 3. Generate a small set of random warehouses
    warehouse_data = []
    for i in range(NUM_WAREHOUSES):
        warehouse_name = f"Warehouse_{i+1}"
        # random city + state combos (just examples)
        possible_locations = [
            "Sao Paulo - SP", "Rio de Janeiro - RJ", "Belo Horizonte - MG",
            "Salvador - BA", "Porto Alegre - RS"
        ]
        warehouse_location = random.choice(possible_locations)
        capacity = random.randint(5000, 20000)  # random capacity
        warehouse_data.append((warehouse_name, warehouse_location, capacity))

    insert_warehouse_sql = """
        INSERT INTO staging.synthetic_warehouse 
        (warehouse_name, warehouse_location, capacity)
        VALUES (%s, %s, %s)
    """
    execute_batch(cursor, insert_warehouse_sql, warehouse_data)

    # 4. Fetch existing orders to create shipments
    #    You can pull from staging.orders OR from dw.fact_orders depending on your pipeline
    fetch_orders_sql = """
        SELECT order_id, order_purchase_timestamp
        FROM staging.orders
        WHERE order_purchase_timestamp IS NOT NULL
        LIMIT 20000  -- example limit if you don't want all
    """
    cursor.execute(fetch_orders_sql)
    orders = cursor.fetchall()  # list of tuples: (order_id, order_purchase_timestamp)

    # 5. Map each existing order to a random warehouse + random ship/delivery
    #    We'll assume shipping always after purchase_date
    #    We'll generate 1 shipment per order for simplicity. 
    #    If you want multiple shipments per order, you can do so.
    shipment_data = []
    for (order_id, purchase_ts) in orders:
        # pick a random warehouse from the newly inserted ones
        warehouse_id = random.randint(1, NUM_WAREHOUSES)  # because synthetic_warehouse_id is 1..N
        carrier = random.choice(CARRIERS)

        # random ship_date: between purchase_ts + 0 hours and purchase_ts + 3 days
        earliest_ship = purchase_ts
        latest_ship = purchase_ts + datetime.timedelta(days=3)
        # but ensure earliest_ship < latest_ship in case times are borderline
        if earliest_ship > latest_ship:
            # swap them if something is off (rare corner case)
            earliest_ship, latest_ship = latest_ship, earliest_ship
        
        ship_date = random_date(earliest_ship, latest_ship)

        # random delivery_date: between ship_date and ship_date + 7 days
        earliest_delivery = ship_date
        latest_delivery = ship_date + datetime.timedelta(days=7)
        delivery_date = random_date(earliest_delivery, latest_delivery)

        shipping_cost = round(random.uniform(5.0, 50.0), 2)  # random cost

        shipment_data.append((
            order_id,
            warehouse_id,
            carrier,
            ship_date,
            delivery_date,
            shipping_cost
        ))

    insert_shipments_sql = """
        INSERT INTO staging.synthetic_shipments
        (order_id, warehouse_id, carrier, ship_date, delivery_date, shipping_cost)
        VALUES (%s, %s, %s, %s, %s, %s)
    """
    execute_batch(cursor, insert_shipments_sql, shipment_data, page_size=1000)

    # 6. Clean up
    cursor.close()
    conn.close()

    print(f"Inserted {len(warehouse_data)} warehouses and {len(shipment_data)} synthetic shipments into staging.")

if __name__ == "__main__":
    main()

Inserted 5 warehouses and 20000 synthetic shipments into staging.


# Method 2: .CSV Output

In [34]:
#!/usr/bin/env python3
"""
generate_synthetic_shipment_data_csv.py

Purpose:
  1. Connects to PostgreSQL to fetch existing order data (order_id, order_purchase_timestamp).
  2. Generates a small set of synthetic warehouse data in a DataFrame.
  3. Creates synthetic shipment records (1 shipment per order) with random carriers, dates, costs.
  4. Writes the results to:
       - synthetic_warehouses.csv
       - synthetic_shipments.csv
  No direct DB insertion—use CSV + COPY from staging approach.

Usage:
  python generate_synthetic_shipment_data_csv.py

After that, from psql or your SQL script, do:
  COPY staging.synthetic_warehouse 
      FROM '/path/to/synthetic_warehouses.csv' CSV HEADER;

  COPY staging.synthetic_shipments 
      FROM '/path/to/synthetic_shipments.csv' CSV HEADER;
"""

import psycopg2
import pandas as pd
import random
import datetime

# -----------------------------------------------------------------------------
# Configurations
# -----------------------------------------------------------------------------
DB_HOST = "localhost"
DB_NAME = "ecommerce_warehouse"
DB_USER = "postgres"
DB_PASSWORD = "YourPasswordHere"
DB_PORT = 5432

NUM_WAREHOUSES = 5
CARRIERS = ["Correios", "Loggi", "Stark Logistics", "Dragonfly Express", "RapidAir"]

OUTPUT_WAREHOUSES_CSV = "synthetic_warehouses.csv"
OUTPUT_SHIPMENTS_CSV = "synthetic_shipments.csv"

# Random date helpers
def random_date(start_date, end_date):
    """
    Return a random datetime between two datetime objects.
    """
    delta = end_date - start_date
    rand_seconds = random.randrange(int(delta.total_seconds()))
    return start_date + datetime.timedelta(seconds=rand_seconds)

def main():
    # 1. Connect to Postgres to fetch orders
    conn = psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        port=DB_PORT
    )
    conn.autocommit = True
    cursor = conn.cursor()

    # For example, pull from staging.orders
    fetch_orders_sql = """
        SELECT order_id, order_purchase_timestamp
        FROM staging.orders
        WHERE order_purchase_timestamp IS NOT NULL
        LIMIT 20000;  -- adjust limit as desired
    """
    cursor.execute(fetch_orders_sql)
    orders = cursor.fetchall()  # list of tuples: (order_id, order_purchase_timestamp)
    cursor.close()
    conn.close()

    print(f"Fetched {len(orders)} orders from staging.orders.")

    # 2. Generate synthetic warehouse data
    warehouse_list = []
    possible_locations = [
        "Sao Paulo - SP",
        "Rio de Janeiro - RJ",
        "Belo Horizonte - MG",
        "Salvador - BA",
        "Porto Alegre - RS"
    ]
    for i in range(1, NUM_WAREHOUSES + 1):
        warehouse_name = f"Warehouse_{i}"
        warehouse_location = random.choice(possible_locations)
        capacity = random.randint(5000, 20000)
        warehouse_list.append({
            "synthetic_warehouse_id": i,  # We'll use 1..N as ID
            "warehouse_name": warehouse_name,
            "warehouse_location": warehouse_location,
            "capacity": capacity
        })

    # Convert to DataFrame for easy CSV output
    df_warehouses = pd.DataFrame(warehouse_list)

    # 3. Create shipments for each order (1:1)
    #    If you want multiple shipments per order, you can create more rows per order.
    shipment_rows = []
    for (order_id, purchase_ts) in orders:
        warehouse_id = random.randint(1, NUM_WAREHOUSES)  # random assignment
        carrier = random.choice(CARRIERS)

        earliest_ship = purchase_ts
        latest_ship = purchase_ts + datetime.timedelta(days=3)
        if earliest_ship > latest_ship:  # rare case if timestamp is weird
            earliest_ship, latest_ship = latest_ship, earliest_ship
        ship_date = random_date(earliest_ship, latest_ship)

        earliest_delivery = ship_date
        latest_delivery = ship_date + datetime.timedelta(days=7)
        delivery_date = random_date(earliest_delivery, latest_delivery)

        shipping_cost = round(random.uniform(5.0, 50.0), 2)

        shipment_rows.append({
            "synthetic_shipment_id": None,  # We'll let DB assign if we want
            "order_id": order_id,
            "warehouse_id": warehouse_id,  # references synthetic_warehouse_id
            "carrier": carrier,
            "ship_date": ship_date,
            "delivery_date": delivery_date,
            "shipping_cost": shipping_cost
        })

    df_shipments = pd.DataFrame(shipment_rows)

    # 4. Write out to CSV
    df_warehouses.to_csv(OUTPUT_WAREHOUSES_CSV, index=False)
    df_shipments.to_csv(OUTPUT_SHIPMENTS_CSV, index=False)

    print(f"Created {OUTPUT_WAREHOUSES_CSV} with {len(df_warehouses)} rows.")
    print(f"Created {OUTPUT_SHIPMENTS_CSV} with {len(df_shipments)} rows.")
    print("Done.")

if __name__ == "__main__":
    main()

Fetched 20000 orders from staging.orders.
Created synthetic_warehouses.csv with 5 rows.
Created synthetic_shipments.csv with 20000 rows.
Done.
